<a href="https://colab.research.google.com/github/Ghanashyam-Bhat/Summer_2/blob/main/CDSAML-PersonalityAnalysisForOnlineProfilles/DatasetGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [274]:
#pip install fer

In [275]:
#pip install image-quality

In [276]:
#from google.colab import drive
#drive.mount('/content/drive')

In [277]:
from PIL import Image
import imquality.brisque as brisque
import cv2 as cv
import numpy as np
import time
from fer import *
import matplotlib.pyplot as plt 
from keras.preprocessing import image
from google.colab.patches import cv2_imshow
%matplotlib inline
import urllib
import requests

from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
# import numpy as np
import argparse
import imutils
# import time
import dlib
# import cv2

In [278]:
def resizing(base,path):
  image_file = Image.open(path)
  image_file.save(base+"edited.jpg", quality=30)

In [279]:
def quality_assesment(base):
  img = Image.open(base+"edited.jpg")
  count_quality = brisque.score(img)
  return count_quality


In [280]:
def face_detection(path2):
  #https://github.com/rk45825243/Face-eye-detection-using-Haar-Cascade-classifier
  img = cv.imread(path2)

  time.sleep(0.2)
  gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

  face_cascade = cv.CascadeClassifier('/content/drive/MyDrive/cdsaml/haarcasscade/haarcascade_frontalface_alt2.xml')
  eye_cascade = cv.CascadeClassifier(r'/content/drive/MyDrive/cdsaml/haarcasscade/haarcascade_eye.xml')
  lefteye_cascade = cv.CascadeClassifier('/content/drive/MyDrive/cdsaml/haarcasscade/haarcascade_lefteye_2splits.xml')
  righteye_cascade = cv.CascadeClassifier('/content/drive/MyDrive/cdsaml/haarcasscade/haarcascade_righteye_2splits.xml')
  faces = face_cascade.detectMultiScale(gray, 1.3, 5)

  count_eyes = 0
  eyes_open = 1

  count_face = 0
  face_areas = []
  ph,pw,pc = img.shape
  pic_area = ph*pw
  eye_area = []
  pic_mid_point = (pw/2,ph/2)
  face_mid_points = []
  #print(ph,pw)
  for (x, y, w, h) in faces:
    cv.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
    roi_gray = gray[y:y + h, x:x + w]
    roi_color = img[y:y + h, x:x + w]
    face_areas.append(w*h)
    face_mid_points.append((x+w/2,y+h/2))
    count_face +=1 
    #print(h,w)

    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex, ey, ew, eh) in eyes:
        cv.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
        count_eyes+=1 

  eye = 0
  openEye = 0
  counter = 0
  openEyes = eye_cascade.detectMultiScale(roi_gray)
  AllEyes = lefteye_cascade.detectMultiScale(roi_gray)
  AllEyes2 = righteye_cascade.detectMultiScale(roi_gray)
  for (ex, ey, ew, eh) in openEyes:
    cv.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0),2)
    openEye += 1

  for (ex, ey, ew, eh) in AllEyes:
    eye += 1
    cv.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 0, 40),2)
    eye_area.append(ew*eh)

  for (ex, ey, ew, eh) in AllEyes2:
    counter += 1
    cv.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 0, 40),2)
    eye_area.append(ew*eh)

  if (openEye != eye):
    print ('Eyes Closed')
    eyes_open = 0


  print(eye,counter,openEye)

  eye_area.sort(reverse=True)
  cv2_imshow( img)
  cv.destroyAllWindows()
  return count_face,count_eyes,face_areas,pic_area,face_mid_points,pic_mid_point,eye_area

In [281]:
def background_detection(path2):
  img = cv.imread(path2)
  #cv2_imshow(img)
  #cv.waitKey(1)

  # Load names of classes and get random colors
  classes = open('/content/drive/MyDrive/cdsaml/darknet/data/coco.names').read().strip().split('\n')
  np.random.seed(42)
  colors = np.random.randint(0, 255, size=(len(classes), 3), dtype='uint8')

  # Give the configuration and weight files for the model and load the network.
  net = cv.dnn.readNetFromDarknet('/content/drive/MyDrive/cdsaml/darknet/cfg/yolov3.cfg', '/content/drive/MyDrive/cdsaml/darknet/yolov3.weights')
  net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
  # net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

  # determine the output layer
  ln = net.getLayerNames()
  #print(net.getUnconnectedOutLayers())
  ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

  # construct a blob from the image
  blob = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
  r = blob[0, 0, :, :]

  #cv2_imshow( r)
  text = f'Blob shape={blob.shape}'
  #cv.displayOverlay('blob', text)
  #cv.waitKey(1)

  net.setInput(blob)
  t0 = time.time()
  outputs = net.forward(ln)
  t = time.time()
  #print('time=', t-t0)

  #print(len(outputs))
  #for out in outputs:
      #print(out.shape)

  # def trackbar2(x):
  #     confidence = x/100
  #     r = r0.copy()
  #     for output in np.vstack(outputs):
  #         if output[4] > confidence:
  #             x, y, w, h = output[:4]
  #             p0 = int((x-w/2)*416), int((y-h/2)*416)
  #             p1 = int((x+w/2)*416), int((y+h/2)*416)
  #             cv.rectangle(r, p0, p1, 1, 1)
  #     #cv.imshow(r)
  #     text = f'Bbox confidence={confidence}'
  #     #cv.displayOverlay('blob', text)

  r0 = blob[0, 0, :, :]
  r = r0.copy()
  #cv2_imshow( r)
  #cv.createTrackbar('confidence', 'blob', 50, 101, trackbar2)
  #trackbar2(50)

  boxes = []
  confidences = []
  classIDs = []
  h, w = img.shape[:2]

  for output in outputs:
      for detection in output:
          scores = detection[5:]
          classID = np.argmax(scores)
          confidence = scores[classID]
          if confidence > 0.5:
              box = detection[:4] * np.array([w, h, w, h])
              (centerX, centerY, width, height) = box.astype("int")
              x = int(centerX - (width / 2))
              y = int(centerY - (height / 2))
              box = [x, y, int(width), int(height)]
              boxes.append(box)
              confidences.append(float(confidence))
              classIDs.append(classID)

  indices = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
  count_objects = 0
  count_person = 0
  if len(indices) > 0:
      for i in indices.flatten():
          (x, y) = (boxes[i][0], boxes[i][1])
          (w, h) = (boxes[i][2], boxes[i][3])
          color = [int(c) for c in colors[classIDs[i]]]
          cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
          text = "{}: {:.4f}".format(classes[classIDs[i]], confidences[i])
          cv.putText(img, text, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
          if "person" not in text:
              count_objects += 1
          else:
              count_person += 1

  cv2_imshow(img)
  cv.waitKey(0)
  cv.destroyAllWindows()
  return count_person,count_objects


In [282]:
def emotion_detection(path2):
  #https://towardsdatascience.com/the-ultimate-guide-to-emotion-recognition-from-facial-expressions-using-python-64e58d4324ff
  test_image_one = plt.imread(path2)
  emo_detector = FER(mtcnn=True)

  # Capture all the emotions on the image
  captured_emotions = emo_detector.detect_emotions(test_image_one)

  # Print all captured emotions with the image
  print(captured_emotions)
  plt.imshow(test_image_one)

  # Use the top Emotion() function to call for the dominant emotion in the image
  dominant_emotion, emotion_score = emo_detector.top_emotion(test_image_one)
  return dominant_emotion,emotion_score,captured_emotions


In [283]:
def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])

    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)

    # return the eye aspect ratio
    return ear


In [284]:

def open_closed_eye(path2):
  frame = cv.imread(path2)
  is_open = 0
  # import the necessary packages
  # frames the eye must be below the threshold
  EYE_AR_THRESH = 0.35
  EYE_AR_CONSEC_FRAMES = 3

  # initialize the frame counters and the total number of blinks
  COUNTER = 0
  TOTAL = 0

  # initialize dlib's face detector (HOG-based) and then create
  # the facial landmark predictor
  print("[INFO] loading facial landmark predictor...")
  detector = dlib.get_frontal_face_detector()
  predictor = dlib.shape_predictor("/content/drive/MyDrive/cdsaml/Dlib/shape_predictor_68_face_landmarks.dat")

  # grab the indexes of the facial landmarks for the left and
  # right eye, respectively
  (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
  (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

  # vs = VideoStream(src=0).start()
  # # vs = VideoStream(usePiCamera=True).start()
  # time.sleep(1.0)

  # loop over frames from the video stream
  #frame = imutils.resize(frame, width=450)
  gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

  # detect faces in the grayscale frame
  rects = detector(gray, 0)
  print(rects)
  # loop over the face detections
  for rect in rects:
      # determine the facial landmarks for the face region, then
      # convert the facial landmark (x, y)-coordinates to a NumPy
      # array
      shape = predictor(gray, rect)
      shape = face_utils.shape_to_np(shape)

      # extract the left and right eye coordinates, then use the
      # coordinates to compute the eye aspect ratio for both eyes
      leftEye = shape[lStart:lEnd]
      rightEye = shape[rStart:rEnd]
      leftEAR = eye_aspect_ratio(leftEye)
      rightEAR = eye_aspect_ratio(rightEye)

      # average the eye aspect ratio together for both eyes
      ear = (leftEAR + rightEAR)

      # compute the convex hull for the left and right eye, then
      # visualize each of the eyes
      leftEyeHull = cv.convexHull(leftEye)
      rightEyeHull = cv.convexHull(rightEye)
      cv.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
      cv.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

      # check to see if the eye aspect ratio is below the blink
      # threshold, and if so, increment the blink frame counter
      if ear < EYE_AR_THRESH:
          cv.putText(frame, "Eye: {}".format("close"), (10, 30),
                  cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
          is_open = 0
          print("Closed Eyes")
          cv.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                  cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)


      # otherwise, the eye aspect ratio is not below the blink
      # threshold
      else:
          cv.putText(frame, "Eye: {}".format("Open"), (10, 30),
                  cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
          is_open = 1
          print("Open Eyes")
          cv.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                  cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

  # draw the total number of blinks on the frame along with
  # the computed eye aspect ratio for the frame

  # show the frame


  # do a bit of cleanup
  cv.destroyAllWindows()
  # vs.stop()
  return is_open

In [285]:
base = "/content/drive/MyDrive/cdsaml/image/"
base2 = "/content/drive/MyDrive/cdsaml/in-the-wild-images"
path = "/content/drive/MyDrive/cdsaml/image/4.jpg"
path2 = base+"edited.jpg"

In [286]:
# for i in range(70):
#   folder = "0"*(2-len(str(i)))+folder+"000"
#   for j in range(1000):
#     file = folder+"0"*(3-len(str(j)))+str(j)
#     path = base+"/"+folder+"/"+file+"png"

In [287]:
# resizing(base,path)
# count_quality = quality_assesment(base)

In [288]:
# count_face,count_eyes,face_areas,pic_area,face_mid_points,pic_mid_point,eye_area = face_detection(path2)
# count_person,count_objects = background_detection(path2)


In [289]:
# dominant_emotion,emotion_score,captured_emotions = emotion_detection(path2)

In [290]:
eyes_open = open_closed_eye(path2)

[INFO] loading facial landmark predictor...
rectangles[[(667, 410) (1438, 1181)]]
Open Eyes


In [291]:
# print("Total number of objects: ",count_objects)
# print("Total number of people: ",count_person)
# print("Total number of face: ",count_face)
# print("Total number of eyes: ",count_eyes)
# print("Image Quality: ",count_quality)
# if count_face>=1:
#   print("Face area ratio: ",max(face_areas)/pic_area)
#   print("Face mid point: ",face_mid_points[face_areas.index(max(face_areas))])
# print("Picture mid point: ",pic_mid_point)
# if len(eye_area)>=2:
#   print("Eye ratio: ",eye_area[1]/eye_area[0])
# print("Emortion: ",dominant_emotion,"(",emotion_score,")")
if eyes_open:
  print("Open Eyes")
else:
  print("Close Eyes")

Open Eyes


In [292]:
# import csv
# with open(base+"dataset.csv","w") as fs:
#   write = csv.writer(fs,delimiter=",")
#   write.writerow(["Count","Quality","Faces","Eyes"])
#   write.writerow([5,count_quality,count_face,count_eyes])